## For this challenge, we will create a program (player function) to play Rock, Paper, Scissors with different opponents. A program that picks at random will usually win 50% of the time. To pass this challenge your program must play matches against four different bots defined below (Quincy, Mrugesh, Kris and Abbey), winning at least 60% of the games in each match.



## The 'player' function will receive an empty string as an argument for the first game in a match since there is no previous play. It is defined with two arguments

## player(prev_play, opponent_history = [ ])

## The function is never called with a second argument so that one is completely optional. The player function contains a second argument (opponent_history = [ ]) to keep track of the previous plays of the opponent between consecutive calls of the player function.

## You only need the opponent_history argument if you want to keep track of the opponent_history.



### Below a play function is defined to play games between two opponents together with the functions that characterize the stragies of 4 available opponents 

In [64]:
#import random


def play(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (
                p1_play == "R" and p2_play == "S") or (p1_play == "S"
                                                       and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return (win_rate)


def quincy(prev_play, counter=[0]):

    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]


def mrugesh(prev_opponent_play, opponent_history=[]):
    
    opponent_history.append(prev_opponent_play)
    
    last_ten = opponent_history[-10:]
    
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == '':
        most_frequent = "S"

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[most_frequent]


def kris(prev_opponent_play):
    if prev_opponent_play == '':
        prev_opponent_play = "R"
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prev_opponent_play]


def abbey(prev_opponent_play,
          opponent_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not prev_opponent_play:    # If the previous play of the opponent is empty string '' it assumes it is 'R 
        prev_opponent_play = 'R'  
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order[0]
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]

### Quick notes about the strategies of the different players:

### Abbey: This is the hardest one, she looks back two last moves of the opponent, find the most frequent pattern of them to develop a counter strategy, Markov Chain Method

### Kris: Depending on the opponents previous play, picks up an ideal response 
### Mrugesh: Looks up the last 10 moves of the opponent and find the most frequent move to oppose it
### Quincy: plays in a cyclic way from the pool of following five choices ["R", "R", "P", "P", "S"], starting from the second entry

### Countering each strategy can be time consuming, below I have taken a hybrid approach: to beat simple stragies like Quincy and Kris, I developed direct counter strategies. To beat Abbey we need to imitate her Markov Chain method, looking back further than her to her previous plays. This automatically helps us to bear Mrugesh as well. In fact, using only this Markov Chain method, we can also beat all the other opponents 

In [65]:
last_five_op = {} # Empty dictonary to be filled with last five plays of the opponent to utilize MC method
def player(prev_play, op_hist=[]):
    
    
    global last_five_op
   
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    
    quincy = ["S", "R", "P", "P"]
    
    kris = ["P", "P", "P", "P"] # kris's moves according our first 4 plays, he will beat us for these plays
    
    choices = ["R", "S", "P"]
    
    
    if prev_play != '': # keep track of the opponent history except ignoring the prev_play in the first round 
                        # which is a empty string
        op_hist.append(prev_play)
        
    #print(op_hist)
    
    first_f = op_hist[:4] # First four elements of the opponents history:
    
    counter = len(op_hist) # counts the number of games played
    
    guess = 'R'
        
    if first_f == quincy and len(op_hist) > 3: # Quincy makes cyclic decisions folling a the pattern
                                                  # above we can easily oppose it using the following line of code
        guess = ideal_response[quincy[counter % len(quincy)]]
    
    if first_f == kris and len(op_hist) > 3:
         
        guess = choices[counter % len(choices)]
    
    # Mrugesh collects data from our last 10 moves and react according to our most frequent plays, given the way we
    # play above we can actually detect if we are playing with Mrugesh or not and then beat him but Abbey is a bigger
    # problem as she uses a sort Markov Chain method looking at our last two moves to predict our next.
    # In particular, she collects the data of our most frequent last two plays in the 'PR', 'RP', 'SS' form,
    # the given our last play from the set : "P" , "R", "S" with our possible next_play = "R/P/S"
    # she is trying the guess our next_play in a statistical manner. 
    # If we want to beat her we should imitate her collecting data from last n moves to construct her next
    # possible reaction in the same manner 
    
    n = 4 # the order we look back will be n+1 
    
    if len(op_hist) > n:
        
        pattern_n = "".join(op_hist[-n:])
        
        if ''.join(op_hist[-(n+1):]) in last_five_op.keys():
            
            last_five_op[''.join(op_hist[-(n + 1):])] += 1
        
        else:
            
            last_five_op[''.join(op_hist[-(n + 1):])] = 1
            
        pot_plays = [pattern_n + 'P', pattern_n + 'R', pattern_n + 'S']
        
        for i in pot_plays:
            if not i in last_five_op.keys():    # If the length 5 pattern is not in our dictionary, sets its value to 0
                last_five_op[i] = 0
        
        sub_order = {k: last_five_op[k]
        for k in pot_plays if k in last_five_op.keys()}
        
        prediction = max(sub_order, key=sub_order.get)[-1:]
        
        guess = ideal_response[prediction]
     
    
    return guess    
    

    

In [67]:
# Before playing with each opponents below, re-run the cells above to define our player function
# and opponent functions. This is becuase our player function and some of the opponent functions save 
# history between consecutive calls, leading to fluctuating results.  

In [50]:
play(player, quincy, 1000)

Final results: {'p1': 994, 'p2': 3, 'tie': 3}
Player 1 win rate: 99.69909729187563%


99.69909729187563

In [52]:
play(player, kris, 1000)

Final results: {'p1': 862, 'p2': 83, 'tie': 55}
Player 1 win rate: 91.21693121693121%


91.21693121693121

In [66]:
play(player, abbey, 1000)

Final results: {'p1': 500, 'p2': 276, 'tie': 224}
Player 1 win rate: 64.43298969072166%


64.43298969072166

In [48]:
play(player, mrugesh, 1000)

Final results: {'p1': 827, 'p2': 162, 'tie': 11}
Player 1 win rate: 83.61981799797775%


83.61981799797775